# Libraries

In [ ]:
!pip install shap
!pip install scikeras
!pip install aif360
!pip install fairlearn

In [ ]:
!pip install aif360[inFairness]
!pip install aif360[OptimalTransport]

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import copy

# Loading Data

In [ ]:
# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
column_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
data = pd.read_csv(url, header=None, names=column_names)
outcomes = data['Outcome']
data_columns = list(data.columns)

In [ ]:
scaler = StandardScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data))
# data_scaled = data_scaled.drop(columns=[8])

In [ ]:
data_pd = pd.concat([data_scaled, outcomes], axis=1)
data_pd = data_pd.rename(columns={0:data_columns[0], 1:data_columns[1], 2:data_columns[2],
                                  3:data_columns[3], 4:data_columns[4], 5:data_columns[5],
                                  6:data_columns[6], 7:data_columns[7]})
data_pd

In [ ]:
min_age = data_pd['Age'].min()
max_age = data_pd['Age'].max()
avg_age = data_pd['Age'].mean()
min_age, max_age, avg_age

### Statistical Analysis

In this section, we thoroughly analyze the values for different features to extract feature-related rules and constraints as well as examining their lower/upper bounds

In [ ]:
data_positive = data_pd[data_pd['Outcome']==1]
data_negative = data_pd[data_pd['Outcome']==0]

In [ ]:
data_positive.columns

In [ ]:
## To check the probabilities for different features, please modify the 'key' variable below,
## as well as the lower and upper bounds for 'pos_freq' and 'neg_freq' variables.

# key = 'Glucose'
# key = 'BloodPressure'
# key = 'SkinThickness'
# key = 'Insulin'
# key = 'BMI'
# key = 'DiabetesPedigreeFunction'
# key = 'Age'
key = 'Pregnancies'

# pos_freq = len(data_positive[key][(data_positive[key]<200) & (data_positive[key]>170)])
# neg_freq = len(data_negative[key][(data_negative[key]<200) & (data_negative[key]>170)])

pos_freq = len(data_positive[key][(data_positive[key]<40) & (data_positive[key]>20)])
neg_freq = len(data_negative[key][(data_negative[key]<40) & (data_negative[key]>20)])

print('Frequency:', pos_freq, neg_freq)
print('Max:',np.max(data_positive[key]), np.max(data_negative[key]))
(pos_freq+1)/(neg_freq+pos_freq+1), (neg_freq+1)/(pos_freq+neg_freq+1)

In [ ]:
## We can illustrate the positive or negative plots either individually or together.
## Plotting such figures can assisst us in determing the rules and lower/upper bounds of the values
## as well as identifying within which range of values the probability of positive/negative labels is higher
## Using such insights, we step by step define and examine the statistical analysis-based constraints

from matplotlib import pyplot as plt

# data_negative[key].plot(label='-')
# data_positive[key].plot(label='+')

data_negative[key].plot(kind='hist', bins=20, label='-')
data_positive[key].plot(kind='hist', bins=20, label='+')
plt.legend()
plt.gca().spines[['top', 'right',]].set_visible(False)

### Dataset Prepation

In [ ]:
# Split data into features and target variable
data_pd = data_pd.drop(8, axis=1)
X = data_pd.drop('Outcome', axis=1)
y = data_pd['Outcome']

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
batch_size = 16

train_dataset = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)))
train_dataset = train_dataset.batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(X_test), tf.convert_to_tensor(y_test)))
test_dataset = test_dataset.batch(batch_size)

# Trivial Learning

In [ ]:
# Define the CNN model
original_model = Sequential([
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

## EarlyStopping
best_loss = float('inf')
best_model_weights = None
patience = 20
val_loss_metric = keras.losses.SparseCategoricalCrossentropy()
early_stopping = True
###

loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = Adam()
n_epochs = 100

val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

for epoch in range(n_epochs):
  print("\nEpoch %d" % (epoch,))
  for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
    with tf.GradientTape() as tape:
        logits = original_model(x_batch_train, training=True)  # Logits for this minibatch
        loss_value = loss_fn(y_batch_train, logits)

    grads = tape.gradient(loss_value, original_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, original_model.trainable_weights))

    if step % batch_size == 0:
        print(
            "Training loss (for one batch) at step %d: %.4f"
            % (step, float(loss_value))
        )
        print("Seen so far: %s samples" % ((step + 1) * batch_size))


  for x_batch_val, y_batch_val in test_dataset:
      val_logits = original_model(x_batch_val, training=False)
      val_acc_metric.update_state(y_batch_val, val_logits)
      val_loss = val_loss_metric(y_batch_val, val_logits)

  val_acc = val_acc_metric.result()
  val_acc_metric.reset_state()
  print("Validation acc: %.4f" % (float(val_acc),))
  print("Validation loss: %.4f" % (float(val_loss),))

  if early_stopping:
    # Early stopping
    if float(val_loss) < best_loss:
        best_loss = float(val_loss)
        best_model_weights = copy.deepcopy(original_model.get_weights())  # Deep copy here
        patience = 20  # Reset patience counter
        print(f"Early Stopping Restart")
    else:
        patience -= 1
        print(f"Early Stopping Patience {patience}")
        if patience == 0:
            break

## Sample Validation Results
# Epoch 49 => Validation acc: 0.7532
# Epoch 99 => Validation acc: 0.7532
# Epoch 49 => Validation acc: 0.7662

## Early Stopping => Epoch 41: # Validation acc: 0.7662  # Validation loss: 0.5262

model_backup = copy.deepcopy(original_model)
if early_stopping:
  original_model.set_weights(best_model_weights)
  print("Best Loss:", best_loss)

### SHAP Value for Normal Learning

In [ ]:
import shap
shap.initjs()

# Calculate SHAP values
## KernelExplainer;
explainer = shap.DeepExplainer(original_model, np.array(X_test))
shap_values = explainer.shap_values(np.array(X_test))
# # Summarize the effects of features
shap.summary_plot(shap_values[:,:,-1], X_test)

In [ ]:
shap.summary_plot(shap_values[:,:,-1], features=X_test, class_names=y_test.unique(), plot_type='bar')

In [ ]:
## The importance values are ordered based on the feature names.
## Meaning: 'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'
importances = []
for i in range(shap_values.shape[1]):
    importances.append(np.mean(np.abs(shap_values[:, i])))
importances

### Fairness

In [ ]:
X_test.columns

In [ ]:
predictions = []
true_labels = []
ages = []

## The age is our protected group.
## Hence, as the fairness metrics are used for binary classification,
## we map the age values using the average age recorded from the dataset
def age_map(i):
    return 0 if i < avg_age else 1

for x_batch_val, y_batch_val in test_dataset:
  pred = constraint_model.predict(x_batch_val)
  predictions.append(tf.argmax(pred, axis=1).numpy())
  true_labels.append(y_batch_val.numpy())

  map_age = np.vectorize(age_map)
  ages.append(map_age(x_batch_val[:,-1].numpy()))

In [ ]:
y_pred = []
y_true = []
feature_ages = []

for age in ages:
    feature_ages.extend(age)

for pred in predictions:
  y_pred.extend(pred)

for label in true_labels:
  y_true.extend(label)

y_pred = np.array(y_pred)
y_true = np.array(y_true)
feature_ages = np.array(feature_ages)

y_pred.shape, y_true.shape, feature_ages.shape

In [ ]:
from fairlearn.metrics import demographic_parity_difference, demographic_parity_ratio

img_preds = y_pred
img_true = y_true

print("Demographic Parity Difference =  ", demographic_parity_difference(img_true, img_preds, sensitive_features=feature_ages))
print("Demographic Parity Ratio = ", demographic_parity_ratio(img_true, img_preds, sensitive_features=feature_ages))


In [ ]:
from fairlearn.metrics import equalized_odds_difference, equalized_odds_ratio
print("Equality Odds Difference =  ", equalized_odds_difference(img_true, img_preds, sensitive_features=feature_ages))
print("Equality Odds Ratio = ", equalized_odds_ratio(img_true, img_preds, sensitive_features=feature_ages))


### AUC and SPD

In [ ]:
from sklearn import metrics
from aif360.sklearn.metrics import statistical_parity_difference

AUC = metrics.roc_auc_score(y_test, y_pred)
print("AUC = ", AUC)

SPD = statistical_parity_difference(pd.DataFrame(y_true), y_pred)
print("SPD = ", SPD)

### Partial Dependence Plots (PDPs)

The PDP plots are another widely mentioned technique for explainability, while produce insights into the partial dependence of different features based on their values.

We haven't used this in reporting the evaluation results, though.

In [ ]:
!pip install scikeras
!pip install scikit-learn==1.2.2

In [ ]:
X_features = list(X_test.columns)

In [ ]:
original_model.dummy_ = "dummy"
from sklearn.utils import validation
validation.check_is_fitted(estimator=original_model)

In [ ]:
from scikeras.wrappers import KerasClassifier
from sklearn.inspection import PartialDependenceDisplay

In [ ]:
kr_original = KerasClassifier(build_fn=original_model, loss=loss_fn, optimizer=optimizer)
kr_original.fit(X_train, y_train, epochs=1)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr_original,
                                        X_test,
                                        features = ['Insulin'],
                                        feature_names = X_features,
                                        ax = ax);

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr_original,
                                        X_test,
                                        features = ['Glucose'],
                                        feature_names = X_features,
                                        ax = ax)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr_original,
                                        X_test,
                                        features = ['Age'],
                                        feature_names = X_features,
                                        ax = ax)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr_original,
                                        X_test,
                                        features = ['Pregnancies'],
                                        feature_names = X_features,
                                        ax = ax)

# Augmented Learning

### Knowledge Definition

In [ ]:
list(data['Pregnancies']).index(5), data_pd['Pregnancies'][list(data['Pregnancies']).index(5)]

In [ ]:
## Constraints
## Original Values => not standardized using Scikit-Learn
# Pregnancies = {'P':[(0, 5, 0.2572178477690289)], 'N':[(0, 5, 0.7427821522309711)]}
# Insulin = {'P':[(200, None, 0.5411764705882353)] , 'N':[(0, 200, 0.7278688524590164)]}
# Glucose = {'P':[(170, None, 0.8405797101449275)], 'N':[(50, 75, 1.0), (30, 100, 0.9270833333333334)]}
# DPF = {'P':[], 'N':[(0, 0.25, 0.7463414634146341)]}
# BMI = {'P':[(50, 60, 0.7142857142857143)], 'N':[(0, 20, 1.0)]}
# BldPrsr = {'P':[(100, None, 0.6153846153846154)], 'N':[]}
# SkinTh = {'P':[(60, None, 1.0)], 'N':[(20, 40, 0.6310975609756098)]}

## Scaled Values => After normalization and standardization using scikit-learn
Pregnancies = {'P':[(-1.141, 0.343, 0.257)], 'N':[(-1.141, 0.343, 0.743)]}
Insulin = {'P':[] , 'N':[(0, 1.0436, 0.7278)]}
Glucose = {'P':[(1.5368, np.inf, 0.8406)], 'N':[(-2.0310, -1.4363, 1.0), (-2.4066, -0.6539, 0.9271)]}
DPF = {'P':[], 'N':[(0, -0.6761, 0.7463)]}
BMI = {'P':[(2.2854, 3.4785, 0.7143)], 'N':[(0, -1.522, 1.0)]}
BldPrsr = {'P':[(1.5971, np.inf, 0.6154)], 'N':[]}
SkinTh = {'P':[(2.475, np.inf, 1.0)], 'N':[(-0.0336, 1.221, 0.6311)]}

constraints = {'Pregnancies':Pregnancies, 'Insulin':Insulin, 'Glucose':Glucose,
               'DiabetesPedigreeFunction':DPF, 'BMI':BMI,
               'BloodPressure':BldPrsr, 'SkinThickness':SkinTh}

In [ ]:
X_train.columns

In [ ]:
## You can either select the features individually or a combination of them using the All Constraints version.
## In case only one or a custom number of constraints are required, you can comment the others in code.

def constraint_evaluator(outputs, features, constraints=constraints):
  preg = 0
  glucose = 0
  insulin = 0
  bmi = 0
  blp = 0
  dpf = 0
  skin = 0

  ## Pregnancies => 0
  preg_bound = 1*(np.less_equal(constraints['Pregnancies']['P'][0][0], features[:,0])*np.less_equal(features[:,0],constraints['Pregnancies']['P'][0][1]))
  preg = preg_bound*((1-outputs)*constraints['Pregnancies']['P'][0][2] + (outputs)*constraints['Pregnancies']['N'][0][2])

  # ## Glucose => 1
  glocuse_bound_p = 1*(np.less_equal(constraints['Glucose']['P'][0][0],features[:,1])*np.less_equal(features[:,1],constraints['Glucose']['P'][0][1]))
  glocuse_bound_n1 = 1*(np.less_equal(constraints['Glucose']['N'][0][0],features[:,1])*np.less_equal(features[:,1],constraints['Glucose']['N'][0][1]))
  glocuse_bound_n2 = 1*(np.less_equal(constraints['Glucose']['N'][1][0],features[:,1])*np.less_equal(features[:,1],constraints['Glucose']['N'][1][1]))

  glucose_p = glocuse_bound_p*(1-outputs)*(constraints['Glucose']['P'][0][2])
  glucose_n = glocuse_bound_n1*outputs*(constraints['Glucose']['N'][0][2]) + glocuse_bound_n2*outputs*(constraints['Glucose']['N'][1][2])
  glucose = glucose_p+glucose_n

  # BloodPressure => 2
  blp_bound = 1*(np.less_equal(constraints['BloodPressure']['P'][0][0], features[:,2])*np.less_equal(features[:,2],constraints['BloodPressure']['P'][0][1]))
  blp = blp_bound*((1-outputs)*constraints['BloodPressure']['P'][0][2])

  # SkinThickness => 3
  skin_bound_p = 1*(np.less_equal(constraints['SkinThickness']['P'][0][0], features[:,3])*np.less_equal(features[:,3],constraints['SkinThickness']['P'][0][1]))
  skin_bound_n = 1*(np.less_equal(constraints['SkinThickness']['N'][0][0], features[:,3])*np.less_equal(features[:,3],constraints['SkinThickness']['N'][0][1]))
  skin = skin_bound_p*(1-outputs)*constraints['SkinThickness']['P'][0][2] + skin_bound_n *(outputs)*constraints['SkinThickness']['N'][0][2]

  # Insuline => 4
  insulin_bound_n = 1*(np.less_equal(constraints['Insulin']['N'][0][0], features[:,4])*np.less_equal(features[:,4], constraints['Insulin']['N'][0][1]))
  insulin = insulin_bound_n*((outputs)*(constraints['Insulin']['N'][0][2]) + (1-outputs)*(1-constraints['Insulin']['N'][0][2]))

  # BMI => 5
  BMI_bound_p = 1*(np.less_equal(constraints['BMI']['P'][0][0], features[:,5])*np.less_equal(features[:,5],constraints['BMI']['P'][0][1]))
  BMI_bound_n = 1*(np.less_equal(constraints['BMI']['N'][0][0], features[:,5])*np.less_equal(features[:,5],constraints['BMI']['N'][0][1]))
  bmi = BMI_bound_p*(1-outputs)*constraints['BMI']['P'][0][2] + BMI_bound_n *(outputs)*constraints['BMI']['N'][0][2]

  ## DiabetesPedigreeFunction => 6
  DPF_bound = 1*(np.less_equal(constraints['DiabetesPedigreeFunction']['N'][0][0], features[:,5])*np.less_equal(features[:,5],constraints['DiabetesPedigreeFunction']['N'][0][1]))
  dpf = DPF_bound*((outputs)*constraints['DiabetesPedigreeFunction']['N'][0][2])


  return sum([preg, insulin, glucose, bmi, blp, dpf, skin])

### Training Loop

In [ ]:
cons_coefficient = 1

# Defining the Diabetes model
constraint_model = Sequential([
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

loss_fn = keras.losses.SparseCategoricalCrossentropy()
def custom_loss_function(logits, x_batch_train, y_batch_train):
   outputs = np.argmax(logits, axis=1)
   const_loss = np.mean(constraint_evaluator(outputs, x_batch_train))
   loss_value = loss_fn(y_batch_train, logits) + cons_coefficient*const_loss
   return loss_value

optimizer = Adam()
n_epochs = 100

## EarlyStopping
best_loss = float('inf')
best_model_weights = None
patience = 20
val_loss_metric = keras.losses.SparseCategoricalCrossentropy()
early_stopping = True
###


val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

for epoch in range(n_epochs):
  print("\nEpoch %d" % (epoch,))
  for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
    with tf.GradientTape() as tape:
        logits = constraint_model(x_batch_train, training=True)  # Logits for this minibatch
        # loss_value = custom_loss_function(logits, x_batch_train, y_batch_train)
        outputs = np.argmax(logits, axis=1)
        const_loss = np.mean(constraint_evaluator(outputs, x_batch_train))
        loss_value = loss_fn(y_batch_train, logits) + cons_coefficient*const_loss

    grads = tape.gradient(loss_value, constraint_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, constraint_model.trainable_weights))

    if step % batch_size == 0:
        print(
            "Training loss (for one batch) at step %d: %.4f"
            % (step, float(loss_value))
        )
        print("Seen so far: %s samples" % ((step + 1) * batch_size))


  for x_batch_val, y_batch_val in test_dataset:
      val_logits = constraint_model(x_batch_val, training=False)
      val_acc_metric.update_state(y_batch_val, val_logits)
      val_loss = val_loss_metric(y_batch_val, val_logits)

  val_acc = val_acc_metric.result()
  val_acc_metric.reset_state()
  print("Validation acc: %.4f" % (float(val_acc),))
  print("Validation loss: %.4f" % (float(val_loss),))

  if early_stopping:
    # Early stopping
    if float(val_loss) < best_loss:
        best_loss = float(val_loss)
        best_model_weights = copy.deepcopy(constraint_model.get_weights())  # Deep copy here
        patience = 20  # Reset patience counter
        print(f"Early Stopping Restart")
    else:
        patience -= 1
        print(f"Early Stopping Patience {patience}")
        if patience == 0:
            break

model_backup = copy.deepcopy(constraint_model)
if early_stopping:
  constraint_model.set_weights(best_model_weights)
  print("Best Loss:", best_loss)


### SHAP Value

In [ ]:
import shap
shap.initjs()

# Calculate SHAP values
## KernelExplainer;
explainer = shap.DeepExplainer(constraint_model, np.array(X_test))
shap_values = explainer.shap_values(np.array(X_test))
## Summarize the effects of features
shap.summary_plot(shap_values[:,:,-1], X_test)

In [ ]:
shap.summary_plot(shap_values[:,:,-1], features=X_test, class_names=y_test.unique(), plot_type='bar')

In [ ]:
importances = []
for i in range(shap_values.shape[1]):
    importances.append(np.mean(np.abs(shap_values[:, i])))
importances

### Partial Dependence Plots (PDPs)

In [ ]:
X_features = list(X_test.columns)

In [ ]:
constraint_model.dummy_ = "dummy"
from sklearn.utils import validation
validation.check_is_fitted(estimator=constraint_model)

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
from scikeras.wrappers import KerasClassifier

In [ ]:
kr = KerasClassifier(build_fn=constraint_model, loss=loss_fn, optimizer=optimizer)
kr.fit(X_train, y_train, epochs=1)

In [ ]:
X_train.columns

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr,
                                        X_test,
                                        features = ['Pregnancies'],
                                        feature_names = X_features,
                                        ax = ax);

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr,
                                        X_test,
                                        features = ['Glucose'],
                                        feature_names = X_features,
                                        ax = ax);

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr,
                                        X_test,
                                        features = ['BloodPressure'],
                                        feature_names = X_features,
                                        ax = ax)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr,
                                        X_test,
                                        features = ['SkinThickness'],
                                        feature_names = X_features,
                                        ax = ax)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr,
                                        X_test,
                                        features = ['Insulin'],
                                        feature_names = X_features,
                                        ax = ax)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr,
                                        X_test,
                                        features = ['BMI'],
                                        feature_names = X_features,
                                        ax = ax)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr,
                                        X_test,
                                        features = ['DiabetesPedigreeFunction'],
                                        feature_names = X_features,
                                        ax = ax)

# Enhanced Augmented Learning

### Knowledge Definition

In [ ]:
list(data['DiabetesPedigreeFunction'])

In [ ]:
i = 0.17
list(data['DiabetesPedigreeFunction']).index(i), data_pd['DiabetesPedigreeFunction'][list(data['DiabetesPedigreeFunction']).index(i)]

In [ ]:
## Constraints
## Original Values
# Pregnancies = {'P':[(0, 5, 0.2572178477690289)], 'N':[(0, 5, 0.7427821522309711)]}
# Insulin = {'P':[(200, None, 0.5411764705882353)] , 'N':[(0, 200, 0.7278688524590164)]}
# Glucose = {'P':[(170, None, 0.8405797101449275)], 'N':[(50, 75, 1.0), (30, 100, 0.9270833333333334)]}
# DPF = {'P':[], 'N':[(0, 0.25, 0.7463414634146341)]}
# BMI = {'P':[(50, 60, 0.7142857142857143)], 'N':[(0, 20, 1.0)]}
# BldPrsr = {'P':[(100, None, 0.6153846153846154)], 'N':[]}
# SkinTh = {'P':[(60, None, 1.0)], 'N':[(20, 40, 0.6310975609756098)]}

## Scaled Values
Pregnancies = {'P':[(-1.141, 0.343, 0.257)], 'N':[(-1.141, 0.343, 0.743)]}
Insulin = {'P':[] , 'N':[(0, 1.0436, 0.7278)]}
Glucose = {'P':[(1.5368, np.inf, 0.8406)], 'N':[(-2.0310, -1.4363, 1.0), (-2.4066, -0.6539, 0.9271)]}
DPF = {'P':[], 'N':[(-1, -0.6761, 0.7463)]}
BMI = {'P':[(2.2854, 3.4785, 0.7143)], 'N':[(-2, -1.522, 1.0)]}
BldPrsr = {'P':[(1.5971, np.inf, 0.6154)], 'N':[]}
SkinTh = {'P':[(2.475, np.inf, 1.0)], 'N':[(-0.0336, 1.221, 0.6311)]}

constraints = {'Pregnancies':Pregnancies, 'Insulin':Insulin, 'Glucose':Glucose,
               'DiabetesPedigreeFunction':DPF, 'BMI':BMI,
               'BloodPressure':BldPrsr, 'SkinThickness':SkinTh}

In [ ]:
X_train.columns

In [ ]:
def constraint_evaluator(outputs, features, weights, meu, constraints=constraints):

  consts_outs = {key: 0 for key in constraints.keys()}

  # ## Pregnancies => 0
  preg_bound = 1*(np.less_equal(constraints['Pregnancies']['P'][0][0], features[:,0])*np.less_equal(features[:,0],constraints['Pregnancies']['P'][0][1]))
  preg = preg_bound*((1-outputs)*constraints['Pregnancies']['P'][0][2] + (outputs)*constraints['Pregnancies']['N'][0][2])
  consts_outs['Pregnancies'] = preg

  # ## Glucose => 1
  glocuse_bound_p = 1*(np.less_equal(constraints['Glucose']['P'][0][0],features[:,1])*np.less_equal(features[:,1],constraints['Glucose']['P'][0][1]))
  glocuse_bound_n1 = 1*(np.less_equal(constraints['Glucose']['N'][0][0],features[:,1])*np.less_equal(features[:,1],constraints['Glucose']['N'][0][1]))
  glocuse_bound_n2 = 1*(np.less_equal(constraints['Glucose']['N'][1][0],features[:,1])*np.less_equal(features[:,1],constraints['Glucose']['N'][1][1]))

  glucose_p = glocuse_bound_p*(1-outputs)*(constraints['Glucose']['P'][0][2])
  glucose_n = glocuse_bound_n1*outputs*(constraints['Glucose']['N'][0][2]) + glocuse_bound_n2*outputs*(constraints['Glucose']['N'][1][2])
  glucose = glucose_p+glucose_n
  consts_outs['Glucose'] = glucose

  # BloodPressure => 2
  blp_bound = 1*(np.less_equal(constraints['BloodPressure']['P'][0][0], features[:,2])*np.less_equal(features[:,2],constraints['BloodPressure']['P'][0][1]))
  blp = blp_bound*((1-outputs)*constraints['BloodPressure']['P'][0][2])
  consts_outs['BloodPressure'] = blp


  # SkinThickness => 3
  skin_bound_p = 1*(np.less_equal(constraints['SkinThickness']['P'][0][0], features[:,3])*np.less_equal(features[:,3],constraints['SkinThickness']['P'][0][1]))
  skin_bound_n = 1*(np.less_equal(constraints['SkinThickness']['N'][0][0], features[:,3])*np.less_equal(features[:,3],constraints['SkinThickness']['N'][0][1]))
  skin = skin_bound_p*(1-outputs)*constraints['SkinThickness']['P'][0][2] + skin_bound_n *(outputs)*constraints['SkinThickness']['N'][0][2]
  consts_outs['SkinThickness'] = skin

  # Insuline => 4
  insulin_bound_n = 1*(np.less_equal(constraints['Insulin']['N'][0][0], features[:,4])*np.less_equal(features[:,4], constraints['Insulin']['N'][0][1]))
  insulin = insulin_bound_n*((outputs)*(constraints['Insulin']['N'][0][2]) + (1-outputs)*(1-constraints['Insulin']['N'][0][2]))
  consts_outs['Insulin'] = insulin


  ## BMI => 5
  BMI_bound_p = 1*(np.less_equal(constraints['BMI']['P'][0][0], features[:,5])*np.less_equal(features[:,5],constraints['BMI']['P'][0][1]))
  BMI_bound_n = 1*(np.less_equal(constraints['BMI']['N'][0][0], features[:,5])*np.less_equal(features[:,5],constraints['BMI']['N'][0][1]))
  bmi = BMI_bound_p*(1-outputs)*constraints['BMI']['P'][0][2] + BMI_bound_n *(outputs)*constraints['BMI']['N'][0][2]
  consts_outs['BMI'] = bmi


  # DiabetesPedigreeFunction => 6
  DPF_bound = 1*(np.less_equal(constraints['DiabetesPedigreeFunction']['N'][0][0], features[:,5])*np.less_equal(features[:,5],constraints['DiabetesPedigreeFunction']['N'][0][1]))
  dpf = DPF_bound*((outputs)*constraints['DiabetesPedigreeFunction']['N'][0][2])
  consts_outs['DiabetesPedigreeFunction'] = dpf

  final_output = {key: 0 for key in constraints.keys()}
  for key in constraints.keys():
    final_output[key] = weights[key]*np.mean(consts_outs[key]) + (meu/2)*(max(0, np.mean(consts_outs[key]))**2)

  return final_output

### Training Loop

In [ ]:
cons_landas = {key: 0 for key in constraints.keys()}
meu = 0.5
C = 0.5
delta = 0.01
threshold = 0.5

# Define the Diabetes model
constraint_model = Sequential([
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = Adam()
n_epochs = 100


## EarlyStopping
best_loss = float('inf')
best_model_weights = None
patience = 20
val_loss_metric = keras.losses.SparseCategoricalCrossentropy()
early_stopping = True
###


val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

for epoch in range(n_epochs):
  print("\nEpoch %d" % (epoch,))
  for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
    with tf.GradientTape() as tape:
        logits = constraint_model(x_batch_train, training=True)  # Logits for this minibatch
        outputs = np.argmax(logits, axis=1)

        theta_loss = 0
        for weight in constraint_model.get_weights():
          theta_loss += delta*tf.nn.l2_loss(weight)

        const_output = constraint_evaluator(outputs, x_batch_train, cons_landas, meu)
        const_loss = sum(list(const_output.values()))

        logit_loss = loss_fn(y_batch_train, logits)
        loss_value = logit_loss + theta_loss + const_loss

    grads = tape.gradient(loss_value, constraint_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, constraint_model.trainable_weights))

    sum_consts = 0
    for key in cons_landas.keys():
      cons_landas[key] += meu*max(0, const_output[key])
      sum_consts += max(0, const_output[key])**2
    # print("Landa", cons_landas)

    if sum_consts > threshold:
      meu *= C

    if step % batch_size == 0:
        print(
            "Training loss, Consts, and Theta loss loss at step %d: %.4f, %.4f, %.4f"
            % (step, float(loss_value), float(const_loss), float(theta_loss))
        )
        print("Seen so far: %s samples" % ((step + 1) * batch_size))
        print("Consts Landas:", cons_landas)


  for x_batch_val, y_batch_val in test_dataset:
      val_logits = constraint_model(x_batch_val, training=False)
      val_acc_metric.update_state(y_batch_val, val_logits)
      val_loss = val_loss_metric(y_batch_val, val_logits)

  val_acc = val_acc_metric.result()
  val_acc_metric.reset_state()
  print("Validation acc: %.4f" % (float(val_acc),))
  print("Validation loss: %.4f" % (float(val_loss),))

  if early_stopping:
    # Early stopping
    if float(val_loss) < best_loss:
        best_loss = float(val_loss)
        best_model_weights = copy.deepcopy(constraint_model.get_weights())  # Deep copy here
        patience = 20  # Reset patience counter
        print(f"Early Stopping Restart")
    else:
        patience -= 1
        print(f"Early Stopping Patience {patience}")
        if patience == 0:
            break

model_backup = copy.deepcopy(constraint_model)
if early_stopping:
  constraint_model.set_weights(best_model_weights)
  print("Best Loss:", best_loss)


### SHAP Value

In [ ]:
import shap
shap.initjs()

# Calculate SHAP values
## KernelExplainer;
explainer = shap.DeepExplainer(constraint_model, np.array(X_test))
shap_values = explainer.shap_values(np.array(X_test))
## Summarize the effects of features
shap.summary_plot(shap_values[:,:,-1], X_test)

In [ ]:
shap.summary_plot(shap_values[:,:,-1], features=X_test, class_names=y_test.unique(), plot_type='bar')

In [ ]:
importances = []
for i in range(shap_values.shape[1]):
    importances.append(np.mean(np.abs(shap_values[:, i])))
importances

### Fairness

In [ ]:
X_test.columns

In [ ]:
predictions = []
true_labels = []
ages = []

def age_map(i):
    return 0 if i < avg_age else 1

for x_batch_val, y_batch_val in test_dataset:
  pred = constraint_model.predict(x_batch_val)
  predictions.append(tf.argmax(pred, axis=1).numpy())
  true_labels.append(y_batch_val.numpy())

  map_age = np.vectorize(age_map)
  ages.append(map_age(x_batch_val[:,-1].numpy()))

In [ ]:
y_pred = []
y_true = []
feature_ages = []

for age in ages:
    feature_ages.extend(age)

for pred in predictions:
  y_pred.extend(pred)

for label in true_labels:
  y_true.extend(label)

y_pred = np.array(y_pred)
y_true = np.array(y_true)
feature_ages = np.array(feature_ages)

y_pred.shape, y_true.shape, feature_ages.shape

In [ ]:
from fairlearn.metrics import demographic_parity_difference, demographic_parity_ratio

img_preds = y_pred
img_true = y_true

print("Demographic Parity Difference =  ", demographic_parity_difference(img_true, img_preds, sensitive_features=feature_ages))
print("Demographic Parity Ratio = ", demographic_parity_ratio(img_true, img_preds, sensitive_features=feature_ages))


In [ ]:
from fairlearn.metrics import equalized_odds_difference, equalized_odds_ratio
print("Equality Odds Difference =  ", equalized_odds_difference(img_true, img_preds, sensitive_features=feature_ages))
print("Equality Odds Ratio = ", equalized_odds_ratio(img_true, img_preds, sensitive_features=feature_ages))


### AUC and SPD

In [ ]:
from sklearn import metrics
from aif360.sklearn.metrics import statistical_parity_difference

AUC = metrics.roc_auc_score(y_test, y_pred)
print("AUC = ", AUC)

SPD = statistical_parity_difference(pd.DataFrame(y_true), y_pred)
print("SPD = ", SPD)

### Partial Dependence Plots (PDPs)

In [ ]:
X_features = list(X_test.columns)

In [ ]:
constraint_model.dummy_ = "dummy"
from sklearn.utils import validation
validation.check_is_fitted(estimator=constraint_model)

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
from scikeras.wrappers import KerasClassifier

In [ ]:
kr = KerasClassifier(build_fn=constraint_model, loss=loss_fn, optimizer=optimizer)
kr.fit(X_train, y_train, epochs=1)

In [ ]:
X_train.columns

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr,
                                        X_test,
                                        features = ['Pregnancies'],
                                        feature_names = X_features,
                                        ax = ax);

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr,
                                        X_test,
                                        features = ['Glucose'],
                                        feature_names = X_features,
                                        ax = ax);

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr,
                                        X_test,
                                        features = ['BloodPressure'],
                                        feature_names = X_features,
                                        ax = ax)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr,
                                        X_test,
                                        features = ['SkinThickness'],
                                        feature_names = X_features,
                                        ax = ax)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr,
                                        X_test,
                                        features = ['Insulin'],
                                        feature_names = X_features,
                                        ax = ax)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr,
                                        X_test,
                                        features = ['BMI'],
                                        feature_names = X_features,
                                        ax = ax)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Partial Dependency Plot")
PartialDependenceDisplay.from_estimator(kr,
                                        X_test,
                                        features = ['DiabetesPedigreeFunction'],
                                        feature_names = X_features,
                                        ax = ax)